# Chat group examples

### SDK option 1

In [ ]:
from promptflow import ChatGroup, ChatAgent

copilot_agent = ChatAgent(name="copilot_flow", flow="./promotflow_copilot")
simulation_agent = ChatAgent(name="simulation_flow", flow="./promptflow_simulation")
# for eager flow, may use below
# simulation_agent = ChatAgent(name="simulation_flow", eager_flow=entry_func)

with ChatGroup(
    max_turns=10,
    max_token=5000,
    max_time=600,  # 10 minutes
    entry_agent=copilot_agent,  # the first agent to speak
    speak_order=[copilot_agent, simulation_agent],  # other than specifying, can also be "LLM" or "AUTO"(default)
) as chat_group:
    copilot_agent.io_mapping(
        question=simulation_agent.outputs.generated_question, 
        model="gpt4",
    )
    simulation_agent.io_mapping(
        persona="Tom", 
        last_answer=copilot_agent.outputs.output,
        chat_history=chat_group.chat_history,  # chat_history is a group-level context
        model="gpt4",
        goal=data_column,  # not sure the original purpose
    )
    # simulation_agent.terminate_func = lambda x: x == "<END>"

    # kick off the chat
    chat_group.run()

    # access to the agent outputs
    print("Chat history:", chat_group.chat_history)
    print("Last copilot flow output:", copilot_agent.outputs["output"])
    print("Last simulation flow output:", simulation_agent.outputs["generated_question"])

### SDK option 2

In [ ]:
from promptflow import ChatGroup, ChatAgent

copilot_agent = ChatAgent(name="copilot_flow", flow="./promotflow_copilot")
simulation_agent = ChatAgent(name="simulation_flow", flow="./promptflow_simulation")

chat_group = ChatGroup(
    agents=[copilot_agent, simulation_agent],
    max_turns=10,
    max_token=5000,
    max_time=600,  # 10 minutes
    entry_agent=copilot_agent,  # the first agent to speak
    speak_order=[copilot_agent, simulation_agent],  # other than specifying, can also be "LLM" or "AUTO"(default)
    io_mapping = {
        "copilot_flow.question": "${simulation_flow.outputs.generated_question}",
        "copilot_flow.model": "gpt4",  # model is an external parameter
        "simulation_flow.persona": "Tom", 
        "simulation_flow.chat_history": "${group.chat_history}",  # chat_history is a group-level context
        "simulation_flow.model": "gpt4",
        "simulation_flow.goal": "<jsonl_data.column1>",  # not sure the original purpose
    }
)

# kick off the chat
chat_group.run()

# access to the agent outputs
print("Chat history:", chat_group.chat_history)
print("Last copilot flow output:", copilot_agent.outputs["output"])
print("Last simulation flow output:", simulation_agent.outputs["generated_question"])


### SDK option 3

In [ ]:
from promptflow import ChatGroup, ChatAgent

copilot_agent = ChatAgent(name="copilot_flow", flow="./promotflow_copilot")
simulation_agent = ChatAgent(name="simulation_flow", flow="./promptflow_simulation")
# for eager flow, may use below
# simulation_agent = ChatAgent(name="simulation_flow", eager_flow=entry_func)

chat_group  = ChatGroup(
    agents=[copilot_agent, simulation_agent],
    max_turns=10,
    max_token=5000,
    max_time=600,  # 10 minutes
    entry_agent=copilot_agent,  # the first agent to speak
    inputs={
        "question": {
            "type": str,
            "default": "What is GPT-4?",
        }
    }
    outputs={
        "output": {
            "type": str,
            "reference": copilot_agent.outputs.output,
        }
    }
)

copilot_agent.set_inputs(
    question=chat_group.inputs.question,
    model="gpt4",
)

simulation_agent.set_inputs(
    persona="Tom", 
    last_answer=copilot_agent.outputs.output,
    chat_history=chat_group.chat_history,  # chat_history is a group-level context
    model="gpt4",
)

# kick off the chat
chat_group.invoke(questioin="What is GPT-4?")

# access to the agent outputs
print("Chat history:", chat_group.chat_history)
print("Last copilot flow output:", copilot_agent.outputs.output.value)
print("Last simulation flow output:", simulation_agent.outputs.generated_question.value)



### SDK Option 4

In [ ]:
from promptflow import chat_group, ChatAgent

copilot_agent = ChatAgent(name="copilot_flow", flow="./promotflow_copilot")
simulation_agent = ChatAgent(name="simulation_flow", flow="./promptflow_simulation")
# for eager flow, may use below
# simulation_agent = ChatAgent(name="simulation_flow", eager_flow=entry_func)

@chat_group(
    max_turns=10,
    max_token=5000,
    max_time=600,  # 10 minutes
    speak_order="definition"
)
def my_chat_group_flow(question: str) -> str:
    # Note:
    #   1. Chat history is no longer group level context, need agent to manage it
    #   2. Need to set entry agent like "copilot.is_entry = True"
    #   3. Cannot use "speak_order" to specify the order of the agents, definition order is the order
    #   4. Cannot specify partial agents to run, or set "copilot_agent.mute = True"
    copilot_agent.set_inputs(
        question=chat_group.inputs.question,  
        model="gpt4",
    )
    copilot_agent.is_entry = True

    simulation_agent.set_inputs(
        persona="Tom", 
        last_answer=copilot_agent.outputs.output,
        chat_history=chat_group.chat_history,  # how to refer to the chat history 
        model="gpt4",
    )

    return {
        "output": copilot_agent.outputs.output,
    }

# kick off the chat
my_chat_group_flow(question="What is GPT-4?")


## Original yaml in experiment
```yaml
# multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 5
    agents:
      - name: copilot_flow
        flow: ../copilot/promotflow_copilot/flow.dag.yaml
        inputs:
          question: ${roles.simulation_flow.outputs.generated_question}
          chat_history: []
          model: ${inputs.model_name}
      - name: simulation_flow
        flow: ../evaluation/similarity.yaml
        inputs:
          persona: ${inputs.persona}
          model: ${inputs.model_name}
          goal: ${data.leo_min_set.query}
          chat_history: ${group.chat_history}
```

### Yaml option 1 - Exp
```yaml
# multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 10
    max_token: 5000
    max_time: 600  # 10 minutes
    entry_agent: copilot_flow
    speak_order: [copilot_flow, simulation_flow]
    agents:
      - name: copilot_flow
        flow: ../copilot/promotflow_copilot/flow.dag.yaml
        inputs:
          question: ${roles.simulation_flow.outputs.generated_question}
          chat_history: []
          model: ${inputs.model_name}
      - name: simulation_flow
        flow: ../evaluation/similarity.yaml
        inputs:
          persona: ${inputs.persona}
          model: ${inputs.model_name}
          chat_history: ${group.chat_history}
        terminate_signal: "<END>"
```

### Yaml option 1 - Flow
```yaml
name: multi_turn_chat
type: chat_group
max_turns: 10
max_token: 5000
max_time: 600  # 10 minutes
inputs:
  question:
    type: str
  persona:
    type: str
  model_name:
    type: str
outputs:
  # chat history output may become built-in
  chat_history:
    type: list
  answer:
    type: str
    reference: ${copilot_flow.outputs.output}
speak_order: [copilot_flow, simulation_flow]
agents:
  - name: copilot_flow
    flow: ../copilot/promotflow_copilot/flow.dag.yaml
    inputs:
      question: ${agents.simulation_flow.outputs.generated_question}
      chat_history: []
      model: ${inputs.model_name}
  - name: simulation_flow
    flow: ../evaluation/similarity.yaml
    inputs:
      persona: ${inputs.persona}
      model: ${inputs.model_name}
      chat_history: ${group.chat_history}
    terminate_signal: "<END>"  # default stop signal
    # terminate_function: "<function source json>"
```

```yaml
agents:
  ...
  - name: simulation_flow
    flow: ../evaluation/similarity.yaml
    inputs:
      persona: ${inputs.persona}
      model: ${inputs.model_name}
      chat_history: ${group.chat_history}
    terminate_signal: "<END>"  # default stop signal
    # terminate_function: "<function source json>"
```

### Yaml option - Inputs/Outputs over conversation history
```yaml
name: multi_turn_chat
type: chat_group
max_turns: 10
inputs:
  topic:
    type: str
    default: movie
    description: "The topic of the conversation"
  model_name:
    type: str
    default: gpt-3.5-turbo
    description: "The model name to use for the conversation"
outputs:
  # chat history output may become built-in output
  chat_history:
    type: list
  result:
    type: str
    reference: ${assistant.outputs.output}
roles:
  - role: assistant
    flow: ./copilot_flow
    inputs:
      question: ${user.outputs.generated_question}
      chat_history: []
      model: ${inputs.model_name}
    initializer:
      question: ${inputs.topic}
  - role: user
    flow: ./simulation_flow
    inputs:
      model: ${inputs.model_name}
      chat_history: ${chat_group.chat_history}
    terminate_signal: "<END>"  # default stop signal
    # terminate_function: "<function source json>"
```

### Yaml option 2

```yaml
# multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 10
    max_token: 5000
    max_time: 600  # 10 minutes
    agents:
      - name: copilot_flow
        flow: ../copilot/promotflow_copilot/flow.dag.yaml
      - name: simulation_flow
        flow: ../evaluation/similarity.yaml
    entry_agent: copilot_flow
    speak_order: [copilot_flow, simulation_flow]
    io_mapping:
        copilot_flow.question: ${simulation_flow.outputs.generated_question}
        copilot_flow.model: ${inputs.model_name}
        copilot_flow.chat_history: []  # maybe omit?
        simulation_flow.persona: ${inputs.persona}
        simulation_flow.chat_history: ${group.chat_history}  # chat_history is a group-level context
        simulation_flow.model: ${inputs.model_name}
        simulation_flow.goal: ${data.leo_min_set.query}  # not sure the original purpose
      
```

### Yaml Option 3 - Conversation history

```yaml
inputs:
  - name: model_name
    type: string
    default: gpt-4
  - name: persona
    type: string
    default: default

data: 
  - name: leo_min_set
    type: jsonl
    path: leo_query_set.jsonl

nodes:
  # multi turn conversation is described as a chat group, which contains a copilot flow and question simulation flow
  - name: multi_turn_chat
    type: chat_group
    max_turns: 5
    stop_signal: "[STOP]"
    roles:
      - #name: copilot_flow
        role: assistant
        path: flow.dag.yaml
        inputs:
          topic: ${data.leo_min_set.query}
          model: ${inputs.model_name}
          conversation_history: ${parent.conversation_history}
      - # name: simulation_flow
        role: user
        path: azureml://registries/azureml/models/similarity/versions/5
        inputs:
          topic: ${data.leo_min_set.query}
          conversation_history: ${parent.conversation_history}

  # evaluate for Leo metrics
  - name: leo_eval
    type: flow
    path: azureml://registries/azureml/models/leo_eval/versions/5
    inputs:
      model: ${inputs.model_name}
      metrics: swiss_leo, ground_leo, pi_leo
      query: ${data.leo_min_set.query}
      answer: ${nodes.multi_turn_chat.conversation_history}
```

### Terminate function

In [ ]:
from promptflow import ChatAgent

def terminate_chat(result):
    # result is the flow result that should be a dict
    return result["score"] >= 10

simulation_agent = ChatAgent(
    name="simulation_flow", 
    flow="./promptflow_simulation",
    terminate_func=terminate_chat,  # set customized terminate function
)

# this is the default terminate function if not specified
def default_terminate_function(result):
    # this would require the flow result has only one output, not a dict
    return result["output"] == "<END>"

In [ ]:
# Option 1
[
    ["copilot_agent", "chat message 1"],
    ["simulation_agent", "chat message 2"],
    ["copilot_agent", "chat message 3"],
]

# Option 2
[
    "<copilot_agent>: chat message 1",
    "<simulation_agent>: chat message 2",
    "<copilot_agent>: chat message 3",
]

# or maybe save both


In [ ]:
from promptflow import trace

In [ ]:
from pathlib import Path
from promptflow._sdk.entities._chat_group._chat_group import ChatGroup
from promptflow._sdk.entities._chat_group._chat_role import ChatRole

TEST_ROOT = Path(r"E:\programs\msft-promptflow\src\promptflow\tests")
FLOWS_DIR = TEST_ROOT / "test_configs/flows"

question = "What's the most beautiful thing in the world?"
ground_truth = "The world itself."

copilot = ChatRole(
    flow=FLOWS_DIR / "chat_group_copilot",
    role="assistant",
    inputs=dict(
        question=question,
        model="gpt-3.5-turbo",
        conversation_history="${parent.conversation_history}",
    ),
)
simulation = ChatRole(
    flow=FLOWS_DIR / "chat_group_simulation",
    role="user",
    inputs=dict(
        question=question,
        ground_truth=ground_truth,
        conversation_history="${parent.conversation_history}",
    ),
)

chat_group = ChatGroup(
    roles=[copilot, simulation],
    max_turns=4,
    max_tokens=1000,
    max_time=1000,
    stop_signal="[STOP]",
)
chat_group.invoke()

In [8]:
a = dict()

print(list(a.values()))


[]


In [ ]:
class ChatGroupOrchestratorProxy:
    def __init__(self, xxx):
        from promptflow._sdk.entities import ChatGroup, ChatRole

        # initialize the chat group object
        self._chat_group_obj = ChatGroup(
            roles=[
                copilot = ChatRole(flow=role_path_1, role="assistant"),
                simulation = ChatRole(flow=role_path_2, role="user"),
            ],
            max_turns=4,
            stop_signal="[STOP]",
        )

    async def exec_line_async(
        self,
        inputs: Mapping[str, Any],
        index: Optional[int] = None,
        run_id: Optional[str] = None,
    ) -> LineResult:
        """Run a single line data through the chat group object."""
        # copy object for concurrent execution
        executed_chat_group_obj = copy.deepcopy(self._chat_group_obj)
        # update inputs
        executed_chat_group_obj._update_role_inputs(inputs)
        # invoke the chat group object
        await executed_chat_group_obj.async_invoke()
        return process_output(executed_chat_group_obj.conversation_history, index, run_id)

# run the batch engine
batch_engine = BatchEngine(xxx)
chat_group_orchestrator_proxy = ChatGroupOrchestratorProxy(xxx)
result = batch_engine.run(xx, executor_proxy=chat_group_orchestrator_proxy)

In [2]:
from pathlib import Path
import json
import tempfile

def merge_jsonl_files(source_folder: str, output_folder: str, group_size: int = 25) -> None:
    source_folder_path = Path(source_folder)
    output_folder_path = Path(output_folder)

    jsonl_files = sorted(source_folder_path.glob('*.jsonl'))

    for i in range(0, len(jsonl_files), group_size):
        group = jsonl_files[i:i+group_size]
        output_file_name = f"{group[0].stem.split('_')[0]}_{group[-1].stem.split('_')[0]}.jsonl"
        output_file_path = output_folder_path / output_file_name

        with output_file_path.open('w') as output_file:
            for jsonl_file in group:
                with jsonl_file.open() as input_file:
                    json_line = json.load(input_file)
                    json.dump(json_line, output_file, ensure_ascii=False)
                    output_file.write('\n')

source_folder = r"C:\Users\hod\.promptflow\.runs\simple_hello_world_variant_0_20240330_152442_732549\flow_artifacts"
output_folder = r"C:\Users\hod\.promptflow\.runs\simple_hello_world_variant_0_20240330_152442_732549\flow_artifacts_merged"
Path(output_folder).mkdir(parents=True, exist_ok=True)
merge_jsonl_files(source_folder, output_folder, 25)


In [1]:
from datetime import datetime

a = datetime.now()
print(a.isoformat())

2024-04-09T15:27:46.470184


: 